## Import

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from functools import reduce

import numpy as np
import pandas as pd
import re, time
import requests
import pickle
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\ipynb\\crawling-code'

In [3]:
out_data_path = 'C:\\Users\\PC0\\Documents\\datasets\\stock_price\\'
data_path = 'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\datasets\\'

* crawling

In [4]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

* agricultural 

In [5]:
start_day = "2012/01/01"

In [6]:
cont = ['germany', 'u.s.', 'brazil', 'switzerland', 'uk', 'italy', 'japan',
       'china', 'canada', 'france', 'south-korea', 'australia']

In [7]:
urls = ['https://kr.investing.com/rates-bonds/{}-1-year-bond-yield-historical-data'.format(x) for x in cont]
urls.extend(['https://kr.investing.com/rates-bonds/{}-10-year-bond-yield-historical-data'.format(x) for x in cont])

In [8]:
names = [(x.split("/")[-1]).split("-year")[0] + "_bond" for x in urls]
names = [x.split("_bond")[0] for x in names]

In [9]:
start_idx = 0

In [10]:
# inner loop
index_list = []

for idx, url in tqdm(enumerate(urls[start_idx:])):
    driver.get(url)
    time.sleep(5)

    # 날짜 입력란
    tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
    tmp_date.click()
    driver.implicitly_wait(3)

    tmp_input = driver.find_element_by_css_selector("input#startDate")
    time.sleep(0.5)

    # 기존 날짜 지우기
    for _ in range(11) :
        tmp_input.send_keys(Keys.BACKSPACE)
        tmp_input = driver.find_element_by_css_selector("input#startDate")
        time.sleep(0.5)

    tmp_input.send_keys(start_day)

    # 요청 버튼
    request_btn = driver.find_element_by_css_selector("a#applyBtn")
    request_btn.click()
    time.sleep(5)

    tables = driver.find_elements_by_css_selector("table#curr_table")

    values = tables[0].text.split("\n")[1:]

    values = [re.sub("[^0-9a-zA-Z.\\- ]","",x) for x in values]

    name = names[idx]

    index_dict = {"date":[],
                 "{}_close".format(name):[],
                 "{}_var".format(name):[]}

    for val in values:
        try :
            year, month, day, close, _, _, _, variation= val.split(" ")
        except ValueError:
            year, month, day, close, _, _, _, vol, variation= val.split(" ")

        date = "-".join([year, month, day])
        index_dict['date'].append(date)
        index_dict['{}_close'.format(name)].append(close)
        index_dict['{}_var'.format(name)].append(variation)

    index_list.append(pd.DataFrame.from_dict(index_dict, orient = "index").T)
    start_idx += 1

index_df = reduce(lambda  left,right: pd.merge(left,right,on=['date'], how='outer'), index_list)

24it [09:18, 23.27s/it]


In [11]:
index_df = index_df.sort_values("date", ascending = False)

In [12]:
index_df.tail()

,date,germany-1_close,germany-1_var,u.s.-1_close,u.s.-1_var,brazil-1_close,brazil-1_var,switzerland-1_close,switzerland-1_var,uk-1_close,...,china-10_close,china-10_var,canada-10_close,canada-10_var,france-10_close,france-10_var,south-korea-10_close,south-korea-10_var,australia-10_close,australia-10_var
2890,2012-01-05,-0.068,21.43,0.1010,-1.94,10.202,0.13,0.250,25.00,0.4030,...,3.450,-1.43,1.972,-1.15,3.384,2.05,3.780,-0.26,3.902,-0.20
2891,2012-01-04,-0.056,229.41,0.1030,-2.83,10.189,0.69,0.200,-83.05,0.3730,...,3.500,-1.41,1.995,0.10,3.316,1.47,3.790,0.00,3.910,-1.09
2892,2012-01-03,-0.017,-76.71,0.1060,-1.85,10.119,0.20,1.180,-4.84,0.3340,...,NaN,NaN,1.993,2.52,3.268,0.93,3.790,0.26,3.953,4.03
2893,2012-01-02,-0.073,-16.09,NaN,NaN,10.099,3.26,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.238,2.73,3.780,0.27,NaN,NaN
3656,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
index_df.shape

(3717, 49)

In [14]:
index_df.to_csv(data_path + "bond_df.csv", index = False)